In [15]:
# 다대다 문자 단위 RNN(Char RNN) - 원핫인코딩

# 모듈 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 데이터 전처리 ###
# 문자열 데이터
input_str = 'apple'
label_str = 'pple!'

# 단어집합
char_vocab = sorted(list(set(input_str+label_str))) # 중복 제거 단어집합
vocab_size = len(char_vocab)
print('문자 집합의 크기 : {}'.format(vocab_size))

# 하이퍼파라미터 초기화
input_size = vocab_size # 입력의 차원
hidden_size = 5 # 은닉상태 크기
output_size = 5 # 출력 크기
learning_rate = 0.1 # 학습률

# 정수 인코딩
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

# 데이터셋 생성
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

# 예측결과 보기 위한 문자 인코딩
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

# 데이터 배치 차원 추가 : nn.RNN()은 기본적으로 3차원 텐서 받기 때문
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

# 데이터 원-핫벡터 변환
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

# 데이터 3D 텐서 변환(RNN 입력을 위함)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

### RNN ###
# RNN셀 구현 - 은닉층 1개 사용
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN셀 구현 - 은닉층 1개 사용
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(input_size, hidden_size, output_size)
outputs = net(X)
print(outputs.shape) # 3차원 텐서
    # RNN셀의 리턴 : 배치 크기, 총 시점 개수, 출력 크기
print(outputs.view(-1, input_size).shape) # 2차원 텐서 변환
print(Y.shape)
print(Y.view(-1).shape) # 1차원 텐서 변환

# 최적화함수, 손실함수 정의
optimizer = optim.Adam(net.parameters(), learning_rate) # 최적화함수 : 아담. 파라미터 기입
criterion = torch.nn.CrossEntropyLoss() # 손실함수 : 크로스엔트로피 함수

문자 집합의 크기 : 5
{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]
[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]
훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])
torch.Size([1, 5, 5])
torch.Size([5, 5])
torch.Size([1, 5])
torch.Size([5])


In [16]:
# 학습
for i in range(100):
    # 최적화함수 초기화
    optimizer.zero_grad()
    
    # 학습
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # batch차원 제거를 위해 np.view 사용
    loss.backward() # 손실함수를 이용한 기울기 계산
    optimizer.step() # 최적화함수를 이용한 파라미터 업데이트
    
    # 모델 예측값 확인
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 timestep별 5차원 벡터에 대해서 가장 높은 값의 인덱스 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)]) # 1인 차원 제거를 위해 np.squeeze 사용
    print(i, "loss : ", loss.item(), "prediction : ", result, "true Y : ", y_data, "prediction str : ", result_str)

0 loss :  1.5796915292739868 prediction :  [[3 3 3 3 3]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  lllll
1 loss :  1.3699228763580322 prediction :  [[4 4 4 4 4]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  ppppp
2 loss :  1.2727556228637695 prediction :  [[4 4 4 4 4]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  ppppp
3 loss :  1.156810998916626 prediction :  [[4 4 4 4 4]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  ppppp
4 loss :  1.0394227504730225 prediction :  [[4 4 4 4 0]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  pppp!
5 loss :  0.9236670732498169 prediction :  [[4 4 4 2 0]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  pppe!
6 loss :  0.8260073661804199 prediction :  [[4 4 4 0 0]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  ppp!!
7 loss :  0.7519238591194153 prediction :  [[4 4 4 0 0]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  ppp!!
8 loss :  0.6750281453132629 prediction :  [[4 4 4 2 0]] true Y :  [[4, 4, 3, 2, 0]] prediction str :  pppe!
9 loss :  0.60817110